In [20]:
import subprocess

class dualfoil:
    
    def __init__(self):
        self.filePath = ''
        self.fileName = 'dualfoil5.in'
        self.vcutL = 1e-4
        self.vcutH = 10.0
        self.first= True
        
        #main output list variables
        time, n_util, p_util, potential, uocp, curr, temp, heatgen = [], [], [], [], [], [], [], []
        
    def __init__(self, vcutL, vcutH):
        self.vcutL = vcutL
        self.vcutH = vcutH
        self.filePath = ''
        self.fileName = 'dualfoil5.in'
        self.first= True
        
    #use when wanting to start a new simulation from scratch
    def reset(self):
        self.first= True
        self.time.clear()
        self.n_util.clear()
        self.p_util.clear()
        self.potential.clear()
        self.uocp.clear()
        self.curr.clear()
        self.temp.clear()
        self.heatgen.clear()
                    
    def run(self):
        if filePath == '':
            subprocess.call('./dualfoil', shell=True)
        else:
            subprocess.call('cd %s && ./dualfoil' %(self.filePath), shell=True)
            
    def set_filepath(path):
        self.filePath = path
            
    def get_voltage(self):
        return potential[-1]

    def get_current(self):
        return curr[-1]
    
    def get_total_time(self):
        rstFile = open('%sdf_restart.dat' %(self.filePath), 'r')
        tmp = rstFile.readline()
        tmp = tmp.lstrip.split()
        #get timestep in minutes
        ts = float(tmp[1]) / 60
        return ts
    
    def get_time_step(self):
        rstFile = open('%sdf_restart.dat' %(self.filePath), 'r')
        tmp = rstFile.readline()
        tmp = tmp.lstrip.split()
        #get timestep in minutes
        ts = float(tmp[0]) / 60
        return ts
    
    #update all list variables so that they represent last-run sim
    def update_output(self):
        t, n, p, v, u, c, tp, hg = extract_main_output()
        self.time.append(t)
        self.n_util.append(n)
        self.p_util.append(p)
        self.potential.append(v)
        self.uocp.append(u)
        self.curr.append(c)
        self.temp.append(tp)
        self.heatgen.append(hg)
    
    #append list variables to the combined output file
    def write_output(self):
        output = [self.time, self.n_util, self.p_util, self.potential, 
                  self.uocp, self.curr, self.temp, self.heatgen]
        subprocess.call('date > %scombinedOutput.out' %(self.filePath), shell=True)

        with open('%scombinedOutput.out' %(self.filePath), 'a') as outFile:
            outFile.write('\nMain Output data\n\n')
            outFile.write('     Time   N_util   P_util Potential    Uocp     Curr    Temp    Heatgen\n')
            outFile.write('     (min)    x         y      (v)        (v)    (A/m2)    (C)     (W/m2)\n\n')
            for i in range(len(time)):
                for j in range(len(output)):
                    outFile.write(str(output[j][i]).rjust(9))
                    outFile.write(',')
                    if j == (len(output)-1):
                        outFile.write('\n')

    def evolve_one_time_step_constant_current(self, time_step, current):
        if self.first== True:
            first_leg(current, time_step, 1)
            self.first = False
        else:
            add_new_leg(current, time_step, 1)
        run()
        update_output()

    def evolve_one_time_step_constant_voltage(self, time_step, voltage):
        if self.first== True:
            first_leg(voltage, time_step, 0)
            self.first = False
        else:
            add_new_leg(voltage, time_step, 0, vcutL, vcutH)
        run()
        update_output()

    def evolve_one_time_step_constant_power(self):
        if self.first== True:
            first_leg(power, time_step, -2)
            self.first = False
        else:
            add_new_leg(power, time_step, -2)
        run()
        update_output()

    def evolve_one_time_step_constant_load(self, time_step, load):
        if self.first== True:
            first_leg(load, time_step, -3)
            self.first = False
        else:
            add_new_leg(load, time_step, -3)
        run()
        update_output()

    def evolve_one_time_step_linear_current(self, time_step, current):
        oldCur = get_current()
        while oldCur <= current:
            if self.first == True:
                first_leg(current, .02/60, 1)
                self.first = False
            else:
                #get next timestep values
                ts = get_time_step()
                add_new_leg(current, ts, 1)
            
            run()
            update_output()
            oldCur = oldCur * 1.1

    def evolve_one_time_step_linear_voltage(self, time_step, voltage):
        oldV = get_voltage()
        while oldV <= voltage:
            if self.first == True:
                #tt(i) value is the number usually assigned as the first timestep value
                first_leg(voltage, .02/60, 1)
                self.first = False
            else:
                #get next timestep values
                ts = get_time_step()
                add_new_leg(voltage, ts, 1)
            
            run()
            update_output()
            oldCur = oldCur * 1.1
    def evolve_one_time_step_linear_power(self, time_step, power):
        return

    def evolve_one_time_step_linear_load(self, time_step, load):
        return
    
    def first_leg(title, cu, tt, mc):    
        
        """
        Resets input file

        Parameters
        ----------
        title : str
            comment that describes the function of the new leg
        cu : float
            defines cu(i) for the input
            if mc = 1 or 2, cu is the current
            if mc = 0, cu is the potential
        tt : float
            defines tt(i) for the input
            if mc = 0 or 1, tt is the new leg's duration (min)
            if mc = 2, tt is the cuttof potential (V)
        mc : float
            defines mc(i) for the input, which controls mode of operation
            possible values for dualfoil 5.2 are 0, 1, and 2
        """
        
        newInput = ''
        tracker = False
        modified = False
        
        s = self.filePath + self.fileName
        with open(s, 'r+') as file:
            line = file.readline()
            while line != '':
                #make sure restart is set to false
                if line.find('.true.') != -1:
                    line = line.replace('.true.', '.false.')

                #find line before the one we need; set tracker 
                if line.find('lcurs') != -1 and modified == False:
                    tmp = line.lstrip().split()
                    #also make sure lcurs is 1
                    if int(tmp[0]) != 1:
                        line = line.replace(str(tmp[0]), '1', 1)
                        newInput += line
                        tracker = True
                        #read past all steps left from previous simulations
                        while line != '\n':
                            line = file.readline()

                if (tracker == True):
                    #replace whatever the current cmd line is with the given leg
                    line = str(cu) + ' ' + str(tt) + ' ' + str(mc) + ' ' + str(self.vcutL) + ' ' + str(self.vcutH) + ' !' + title + '\n\n'
                    tracker = False 
                    modified = True    

                #keep up the new file and read next line
                newInput += line  
                line = file.readline()

        with open(s, 'w') as file:
            file.write(newInput)

    def add_new_leg(title, cu, tt, mc):

        """
        Appends a new leg to dualfoil's input file, adjusting the restart parameter if needed.
        Also edits previous leg as needed to avoid data errors.

        Parameters
        ----------
        title : str
            comment that describes the function of the new leg
        cu : float
            defines cu(i) for the input
            if mc = 1 or 2, cu is the current
            if mc = 0, cu is the potential
        tt : float
            defines tt(i) for the input
            if mc = 0 or 1, tt is the new leg's duration (min)
            if mc = 2, tt is the cuttof potential (V)
        mc : float
            defines mc(i) for the input, which controls mode of operation
            possible values for dualfoil 5.2 are 0, 1, and 2
        """

        newInput = ''
        newLeg = ''
        tracker = False
        modified = False
        firstRst = False
        
        s = self.filePath + self.fileName
        with open(s, 'r+') as file:
            line = file.readline()
            while line != '':
                #make sure restart is set to true
                if line.find('.false.') != -1:
                    firstRst = True
                    line = line.replace('.false.', '.true.')


                if (tracker == True):
                    #if we have already made all required modifications, add new leg
                    if (modified == True):
                        newLeg = str(cu) + ' ' + str(tt) + ' ' + str(mc) + ' ' + str(self.vcutL) + ' ' + str(self.vcutH) + ' !' + title + '\n'
                        newInput += newLeg
                        tracker = False 
                    else: 
                        #Make required modifications; should be left with a reference leg and the new leg.

                        #Get the total simulation time; we might need it
                        totalT = get_total_time()

                        #if there are 2 legs from a previous restart, only keep the last one
                        if firstRst == False: 
                            line = file.readline()
                            #^this leg^ will become the reference leg for the next simulation

                            #modify tt(i) of most recent leg if it is terms of time
                            tmp = line.split(' ')

                            #check mc(i): did most recent leg depended on time?
                            if float(tmp[2]) == 1 or float(tmp[2]) == 0:
                                #replace old tt(i) with total run time so far
                                oldTT = float(tmp[1])
                                line = line.replace(str(oldTT), str(totalT))

                                #mark that we have completed tasks
                                modified = True

                #find line before the one we need; set tracker for next loopthru
                if line.find('lcurs') != -1 and modified == False:
                    #if first restart, must change lcurs to 2 steps
                    if firstRst == True:
                        line = line.replace('1', '2', 1)
                        tracker = True

                #keep up the new file and read next line
                newInput += line  
                line = file.readline()

        with open(s, 'w') as file:
            file.write(newInput)

def extract_main_output():
    """
    Returns
    -------
    time : list of float
    n_util : list of float
    p_util : list of float
    potential : list of float
    uocp : list of float
    curr : list of float
    temp : list of float
    heatgen : list of float
    """

    #first go through and find position where output starts in file
    x = 0
    previous = ''
    with open('%sdualfoil5.out' %(self.filePath), 'r') as fin:
        data_list = []

        for line in fin.readlines():
            if line.find('(min)') != -1:
                #found it! stop here
                break
                x += 1

    #now read lines again 
    with open(file, 'r') as fin:

        for line in fin.readlines()[x+2:] :
            #only take lines with convertable data
            if line.find(',') != -1:
                #make sure we are not taking in a copy 
                if line != previous:
                    previous = line
                    line = line.rstrip('\n').rstrip(' ').lstrip(' ')
                    data_list.append(line)  

    #variable lists for each time
    time = [];
    n_util = []
    p_util = []
    potential = []
    uocp = []
    curr = []
    temp = []
    heatgen = []

    for data in data_list:
        tmp = data.split(',')
        for i in tmp:
            i.lstrip(' ')
            time.append(float(tmp[0]))
            n_util.append(float(tmp[1]))
            p_util.append(float(tmp[2]))
            potential.append(float(tmp[3]))
            uocp.append(float(tmp[4]))
            curr.append(float(tmp[5]))
            temp.append(float(tmp[6]))
                                                                                                                               
            #for 5.1 code                                                                                                                    
            if (tmp[7] == ' ******'):
                tmp[7] = '0.00'   
                heatgen.append(float(tmp[7]))

    #return data in order it appears
    return time, n_util, p_util, potential, uocp, curr, temp, heatgen

